In [0]:
%sql
DROP volume IF EXISTS delta.autoloader.batch19;




In [0]:
%sql
CREATE VOLUME delta.autoloader.batch19;


In [0]:
%sql
drop table if exists delta.autoloader.transactions_bronze_19

In [0]:
df = (spark.readStream
          .format("cloudFiles")
          .option("cloudFiles.format", "csv")
          .option("header", "true")
          .option("cloudFiles.inferColumnTypes", "true")
          .option("cloudFiles.schemaLocation", "/Volumes/workspace/default/batch19/_schema")
          .option("cloudFiles.schemaEvolutionMode", "addNewColumns")
          .option("cloudFiles.maxFilesPerTrigger","1")
          .load("/Volumes/delta/autoloader/batch19"))

In [0]:

df.writeStream\
   .format("delta")\
   .option("checkpointLocation", "/Volumes/delta/autoloader/batch19/_checkpoints")\
   .option("mergeSchema", "true")\
   .outputMode("append")\
   .trigger(once=True)\
   .table("delta.autoloader.transactions_bronze_19")


In [0]:
%sql
select count(*) from delta.autoloader.batch19

In [0]:
df_csv = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("/Volumes/delta/autoloader/batch19/sales-data-sample.csv")

display(df_csv)

In [0]:
display(df_parquet_clean.count())

In [0]:
df_parquet = spark.read.format("delta").load(
    "/Volumes/delta/autoloader/batch19/sink"
)
display(df_parquet)

In [0]:
df_parquet_clean = (
    df_parquet
    .filter("CustomerName = 'Toby Swindell'")
    .dropna()
)
display(df_parquet_clean)

In [0]:
df_parquet_clean = (
    df_parquet
    .filter("CustomerName != 'Toby Swindell' OR (CustomerName = 'Toby Swindell' AND NOT (CustomerName IS NULL OR OrderID IS NULL OR ProductName IS NULL))")
)
display(df_parquet_clean)